In [181]:
# Import necessary libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

import tensorflow as tf

In [182]:
# Hyperparameters data loading

base_file_path = 'C:/Users/nikoLocal/Documents/Opencampus/Machine_Vision_challenge_data/'
image_path = base_file_path + '/input_train/input_train'

label_csv_name = 'Y_train_eVW9jym.csv'


In [183]:
# Load the data and briefly inspect

# load data from csv to pandas dataframe
train_df = pd.read_csv(os.path.join(base_file_path, label_csv_name))

train_df.head()

train_df.duplicated(subset='filename').value_counts()
#result: no duplicated file names


False    8278
Name: count, dtype: int64

In [184]:
#ToDo - assign each label a number according to the website so that there is no confusion as to what is what!

#add another column to the dataframe according

dict_numbers = {'GOOD': 0,'Boucle plate':1,'Lift-off blanc':2,'Lift-off noir':3,'Missing':4,'Short circuit MOS':5}
dict_strings = {'GOOD': '0_GOOD','Boucle plate':'1_Flat loop','Lift-off blanc':'2_White lift-off','Lift-off noir':'3_Black lift-off','Missing':'4_Missing','Short circuit MOS':'5_Short circuit MOS'}

label_list = ['4_Missing','1_Flat loop','2_White lift-off','0_GOOD','3_Black lift-off','5_Short circuit MOS']
label_list.sort()

train_df['LabelNum'] = train_df['Label'].map(dict_numbers)
train_df['LabelStr'] = train_df['Label'].map(dict_strings)

In [185]:
train_df.head()

,Unnamed: 0,filename,window,lib,Label,LabelNum,LabelStr
0,0,15b3bab7c186fd35b65df777890c427dd243feacbb85dd...,2003,Die01,Missing,4,4_Missing
1,1,1856617e1ac2d821a46a41b938818f0169342226a78f93...,2003,Die01,GOOD,0,0_GOOD
2,2,19066cce773b3a092ebf4311b11858aa653da6f8274957...,2003,Die01,Missing,4,4_Missing
3,3,19c10caf4b24284e1748caed62d94cbb689d6b379b1cf5...,2003,Die01,GOOD,0,0_GOOD
4,4,1a627426d55a668df8bcd381a7fa87b620481995b6755f...,2003,Die01,Missing,4,4_Missing


In [186]:
# Take dataframe subsets for specific tasks, i.e. balanced datasets

#Hyperparameters:
test_split = 0.2
val_split = 0.2 # remember - this is fractional  after the test data has been split from the initial balanced sub dataset

# list of Labels = Missing  , GOOD  , Lift-off blanc  , Short circuit MOS   ,  Lift-off noir   , Boucle plate
#included_labels = ['GOOD', 'Missing','Lift-off blanc','Short circuit MOS','Lift-off noir','Boucle plate']
included_labels = ['0_GOOD','1_Flat loop','2_White lift-off','3_Black lift-off','4_Missing','5_Short circuit MOS']
num_classes = len(included_labels)

# get counts of label with the least entries
countList = train_df['LabelStr'].value_counts()
minCounts = countList[included_labels].min()

BalancedDF = pd.DataFrame()

#concat sampled dataframes for each included label
for i in range(num_classes):
    BalancedDF = pd.concat([BalancedDF,train_df[train_df['LabelStr'] == included_labels[i]].sample(n=minCounts)],axis=0)

#test if worked as intended
BalancedDF['LabelStr'].value_counts()

#make train-test split using sklean function
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

#split dataframe according to fractional test size
train_df_balanced, test_df_balanced = train_test_split(BalancedDF, test_size=test_split, random_state=42) #keep random state constant to ensure comparability



In [187]:
#test how str are sorted automatically



In [188]:
train_df_balanced['Label'].value_counts()

Label
Missing              62
Lift-off noir        58
Lift-off blanc       57
Short circuit MOS    56
GOOD                 54
Boucle plate         53
Name: count, dtype: int64

In [189]:
# initialize ImageDataGenerators
# use ImageDataGen because it has method flow_from_dataframe() that works really well together with pandas dataframes
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
# although deprecated the functionality can be used as discussed in feedback session

from keras.src.legacy.preprocessing.image import ImageDataGenerator

# HYPERPARAMTERS ########

target_size = (256,256) #pixel size to load img
class_mode = 'categorical' # how to store labels - either categorical (one-hot encoding) or as numbers
#class_mode = 'input'
batch_size = 64 #later player around with batch size to see how it affects performance
labelCol = 'LabelStr'
#########################

#normalize pixel intensities
rescale = 1.0/255.0

datagen = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=val_split)

datagen_augmentation = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=35,
    shear_range= 12,
    zoom_range = 0.10,
    rescale=rescale,
    validation_split=val_split)

datagen_test = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=0.0)

train_generator = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_val = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_aug = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_aug_val = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_metrics = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=False,
    seed=42,
    subset='training')

#why are only 6209 file names found? - because of subset!


Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.


In [190]:
# build a model to be used as baseline model
# use "simplest" CNN as baseline

model_simple_CNN = tf.keras.Sequential([
    tf.keras.layers.Input((target_size[0], target_size[1], 1)),  #image are greyscale - so in total dim (width,height,1)
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model_simple_CNN.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 254, 254, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │    31,490,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,509,766 (120.20 MB)

 Trainable params: 31,509,766 (120.20 MB)

 Non-trainable params: 0 (0.00 B)

In [191]:
# compile Model

model_simple_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    metrics=["accuracy",'precision','recall'],
)

In [192]:
# To Do - set stopping criteria via callbacks. Either based on accuracy or decreasing changes in validation loss

# callback that monitors validation accuracy / loss
# https://keras.io/api/callbacks/early_stopping/
val_loss_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose = 2
)


In [ ]:
history = model_simple_CNN.fit(
    train_generator_aug,
    validation_data = train_generator_val,
    epochs=50,
    callbacks=[val_loss_stop],
    verbose = 2 #2 is one line per epch
)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.2132 - loss: 7.2021 - precision: 0.1905 - recall: 0.1029 - val_accuracy: 0.1912 - val_loss: 3.3618 - val_precision: 0.4333 - val_recall: 0.1912
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2022 - loss: 2.8213 - precision: 0.3333 - recall: 0.0478 - val_accuracy: 0.2794 - val_loss: 2.0113 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.3015 - loss: 1.8479 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1029 - val_loss: 1.7816 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2390 - loss: 1.7521 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2206 - val_loss: 1.7805 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2831 - loss: 1.7349 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy

In [153]:
# test accuracy on test data

test_loss, test_accuracy = model_simple_CNN.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy} | Test Loss: {test_loss}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.8605 - loss: 0.6225
Test Accuracy: 0.8604651093482971 | Test Loss: 0.6225393414497375


In [160]:
#get predicted and true labels for the test dataset

#get true labels from generator:
#  https://stackoverflow.com/questions/45413712/keras-get-true-labels-y-test-from-imagedatagenerator-or-predict-generator

from keras.utils import to_categorical

# test_generator.labels and test_generator.classes seem to be identical
#test_generator_metrics.classes

true_labels = test_generator_metrics.classes
#make into one hot encoding
#true_labels = to_categorical(true_labels, num_classes=6)

# model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
predicted_labels = model_simple_CNN.predict(test_generator_metrics)

#convert to numerical - np.argmax directly does the job
predicted_labels = np.argmax(predicted_labels, axis=-1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


In [155]:
#true_labels[:10]

In [156]:
#test_df_balanced.head(10)

In [157]:
# IMPORTANT
# ImageDataGen and FlowFromDataFrame make numerical (categorical) labels from the Str labels by sorting them numerically.
# Thus, by sorting  the labels by hand via numbers in front we can guarantee the correct numerical label outputs matching the challenge online

In [158]:
#use more sophisticated metrics
from sklearn.metrics import classification_report

#classification report does NOT work with one hot encoding
print(classification_report(true_labels, predicted_labels,target_names = label_list))

                     precision    recall  f1-score   support

             0_GOOD       0.85      1.00      0.92        17
        1_Flat loop       0.77      0.67      0.71        15
   2_White lift-off       0.75      0.64      0.69        14
   3_Black lift-off       0.89      0.89      0.89         9
          4_Missing       0.90      1.00      0.95        18
5_Short circuit MOS       1.00      0.92      0.96        13

           accuracy                           0.86        86
          macro avg       0.86      0.85      0.85        86
       weighted avg       0.86      0.86      0.86        86



In [159]:
'''
from sklearn.utils import class_weight as cw

def get_weight(y):
    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current

a = get_weight(train_generator.classes)
'''

"\nfrom sklearn.utils import class_weight as cw\n\ndef get_weight(y):\n    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)\n    return class_weight_current\n\na = get_weight(train_generator.classes)\n"